In [3]:
import psycopg2
from psycopg2.extras import execute_values
from tqdm import tqdm

# Function to get the connection and cursor
def get_conn():
    conn = psycopg2.connect(database="postgres",
                            host="localhost",
                            user="postgres",
                            password="mysecretpass",
                            port="5432")
    cursor = conn.cursor()
    return cursor, conn

# Function to insert data in bulk
def insert_data_in_bulk(data):
    cursor, conn = get_conn()
    insert_statement = f"INSERT INTO trades (country_code, year, comm_code, flow, trade_usd, kg, quantity, quantity_name) VALUES {data};"
    cursor.execute(insert_statement)
    conn.commit()
    conn.close()

# Read data from the SQL file
with open('postgres_public_trades.sql', 'r') as file:
    data = file.read()

def split_into_batches(data, num_items):
    data_list = data.split(',\n')
    batches = [',\n'.join(data_list[i:i + num_items]) for i in range(0, len(data_list), num_items)]
    return batches

num_items = 10000  # Specify the number of items per batch
batches = split_into_batches(data, num_items)

# Call the function for each batch with a progress bar
for i, batch in enumerate(tqdm(batches, desc="Inserting data")):
    insert_data_in_bulk(batch)


Inserting data: 100%|██████████| 622/622 [02:14<00:00,  4.63it/s]
